In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
        
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
notebook_path = os.path.dirname(os.path.abspath("v4.ipynb"))+"\\"
input_path = notebook_path + 'input\\'

In [3]:
train = pd.read_csv(input_path + 'train.csv', index_col='id')
test = pd.read_csv(input_path + 'test.csv', index_col='id')
train, X_test, y_train, y_test = train_test_split(train, train['target'], train_size=0.60)
X_test.drop(columns=['target'], inplace=True)

In [4]:
for i in list(train.index):
    if train['target'][i] < 5.5 or train['target'][i] > 10:
        train.drop(i, inplace=True)

In [5]:
y_train = train.pop("target")
X_train = train

In [6]:
import xgboost as xgb

objective ='reg:squarederror', reg_alpha=0.94, booster='dart', eta = 0.2, eval_metric = 'rmse', tree_method='hist'

In [8]:
params={
 'n_estimators': 300,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'eta': 0.03,
 'max_depth': 11,
 'min_child_weight': 10,
 'tree_method':'gpu_hist'
}
model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.699136


In [9]:
preds = model.predict(test)
my_df = pd.DataFrame(list(test.index), columns=['id'])
my_df['target'] = preds
my_df.to_csv('sub.csv', index=False)

Лучшая модель:

In [10]:
params={
 'objective': 'reg:squarederror',
 'reg_alpha': 0.94,
 'booster': 'dart',
 'eval_metric': 'rmse',
 'n_estimators': 300,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'eta': 0.03,
 'max_depth': 11,
 'min_child_weight': 10,
 'tree_method':'gpu_hist'
}
model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.698928


In [11]:
preds = model.predict(test)
my_df = pd.DataFrame(list(test.index), columns=['id'])
my_df['target'] = preds
my_df.to_csv('sub.csv', index=False)

In [27]:
params={
 'objective': 'reg:squarederror',
 'reg_alpha': 0.95,
 'reg_lambda': 0.05,
 'booster': 'dart',
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'eval_metric': 'rmse',
 'eta': 0.035,
 'max_depth': 12,
 'min_child_weight': 20,
 'tree_method': 'gpu_hist',
 'n_estimators': 300
}
model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.699356


Попытка поработать с рандомным лесом, но безуспешная, для того, чтобы добиться хорошего результата, моих вычислительных мощностей недостаточно

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import pydot

regr = RandomForestRegressor(n_estimators = 500, max_depth=15, criterion='mse', min_samples_split=500, max_features='sqrt')
regr.fit(X_train, y_train)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

tree = regr.estimators_[5]
export_graphviz(tree, out_file = 'tree1.dot', feature_names = list(X_train.columns), rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('tree1.dot')
graph.write_png('tree1.png')

from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 15],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 5],
    'min_samples_split': [8, 10],
    'n_estimators': [100, 200]
}

rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

grid_search.best_params_

best_grid = grid_search.best_estimator_
preds = best_grid.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = .predict(test)
my_df = pd.DataFrame(list(test.index), columns=['id'])
my_df['target'] = preds
my_df.to_csv('sub.csv', index=False)